# Sympy's Symbolic Quantum Mechanics Package

SymPy has a subpackage, `sympy.physics.quantum` implements a general symbolic QM. We mix a standard QM and Q information/computing (QC) as below.

QM Foundations 
- dirac notation.ipynb (QM – absolutely fundamental formalism)
- density.ipynb (QM – mixed states, statistical interpretation)
- angular_momentum.ipynb (QM – core operator, must-know)
- spin_orbit interaction.ipynb (QM – physical coupling, often tested)
- sho1d.ipynb (QM – harmonic oscillator, cornerstone model)

QC Core Tools 
- qubits.ipynb (QM/QC bridge – two-level systems, essential for both)
- decompose.ipynb (QC – gate decomposition, useful for implementation)
- fidelity.ipynb (QC – measure of state overlap, key in protocols)
- teleportation.ipynb (QC – fundamental protocol, conceptually important)
- dense coding.ipynb (QC – paired with teleportation, shows entanglement power)
- error_correction.ipynb (QC – crucial for real systems, higher-level)
- qft.ipynb (QC – advanced, building block for Shor’s algorithm, etc.)
- grovers.ipynb (QC – advanced, canonical quantum speedup algorithm)

Example: using `sympy.physics.quantum` to create a 3 qubit [Quantum Fourier Transform](https://en.wikipedia.org/wiki/Quantum_Fourier_transform), decompose the circuit into primitive gates, and then visualize the circuit:

<img src="https://raw.githubusercontent.com/tomctang/NB_img/main/qft_example.png" alt="exp.1" width="500">


# Basic Symbolic QM - dirac

In [1]:
from sympy import init_printing
init_printing(use_latex=True)

In [2]:
from sympy import sqrt, symbols, Rational, srepr
from sympy import expand, Eq, Symbol, simplify, exp, sin
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import *
from sympy.physics.quantum.gate import *
from sympy.physics.quantum.grover import *
from sympy.physics.quantum.qft import QFT, IQFT, Fourier
from sympy.physics.quantum.circuitplot import circuit_plot

## Bras and kets

Symbolic kets can be created using the `Ket` class as seen here:

In [3]:
phi, psi = Ket('phi'), Ket('psi')

These ket instances are fully symbolic and behave exactly like the corresponding mathematical entities.
For example, one can form a linear combination using addition and scalar multiplication:

In [4]:
alpha = Symbol('alpha', complex=True)
beta = Symbol('beta', complex=True)
state = alpha*psi + beta*phi; state

Bras can be created using the `Bra` class directly or by using the `Dagger` class
on an expression involving kets:

In [5]:
ip = Dagger(state)*state; ip

Because this is a standard SymPy expression, we can use standard SymPy functions and methods
for manipulating expression. Here we use expand to multiply this expression out, followed
by `qapply` which identifies inner and outer products in an expression.

In [6]:
qapply(expand(ip))

## Operators

SymPy also has a full set of classes for handling symbolic operators.  Here we create three operators,
one of which is hermitian:

In [7]:
A = Operator('A')
B = Operator('B')
C = HermitianOperator('C')

When used in arithmetic expressions SymPy knows that operators do not commute under
multiplication/composition as is seen by expanding a polynomial of operators:

In [8]:
expand((A+B)**2)

Commutators of operators can also be created:

In [9]:
comm = Commutator(A*B,B+C); comm

The `expand` function has custom logic for expanding commutators using standard commutator
relations:

In [10]:
comm.expand(commutator=True)

Any commutator can be performed ($[A,B]\rightarrow AB-BA$) using the `doit` method:

In [11]:
_.doit().expand()

The `Dagger` class also works with operators and is aware of the properties of unitary
and hermitian operators:

In [12]:
Dagger(_)

## Tensor products

Symbolic tensor products of operators and states can also be created and manipulated:

In [13]:
op = TensorProduct(A,B+C)
state = TensorProduct(psi,phi)
op*state

Once a tensor product has been created, it can be simplified,

In [14]:
tensor_product_simp(_)

/tmp/ipykernel_72469/1014556276.py:1: SymPyDeprecationWarning: 

tensor_product_simp has been deprecated. The transformations
performed by this function are now done automatically when
tensor products are combined.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-tensorproduct-simp
for details.

This has been deprecated since SymPy version 1.14. It
will be removed in a future version of SymPy.

  tensor_product_simp(_)


and expanded:

In [15]:
expand(_)

# Density Operator and Matrix 

In [ ]:
from IPython.display import display

In [ ]:
# TODO: there is a bug in density.py that is preventing this from working, uncomment to reproduce
# from sympy import init_printing
# init_printing(use_latex=True)

In [ ]:
from sympy import *
from sympy.core.trace import Tr
from sympy.physics.quantum import *
from sympy.physics.quantum.density import *
from sympy.physics.quantum.spin import (
    Jx, Jy, Jz, Jplus, Jminus, J2,
    JxBra, JyBra, JzBra,
    JxKet, JyKet, JzKet,
)

## Basic density operator

Create a density matrix using symbolic states:

In [ ]:
psi = Ket('psi')
phi = Ket('phi')

In [ ]:
d = Density((psi,0.5),(phi,0.5));
d

In [ ]:
d.states()

In [ ]:
d.probs()

In [ ]:
d.doit()

In [ ]:
Dagger(d)

In [ ]:
A = Operator('A')

In [ ]:
d.apply_op(A)

## Density operator for spin states

Now create a density operator using spin states:

In [ ]:
up = JzKet(S(1)/2,S(1)/2)
down = JzKet(S(1)/2,-S(1)/2)

In [ ]:
d2 = Density((up,0.5),(down,0.5)); d2

In [ ]:
represent(d2)

In [ ]:
d2.apply_op(Jz)

In [ ]:
qapply(_)

In [ ]:
qapply((Jy*d2).doit())

## Evaluate entropy of the density matrices

In [ ]:
entropy(d2)

In [ ]:
entropy(represent(d2))

In [ ]:
entropy(represent(d2,format="numpy"))

In [ ]:
entropy(represent(d2,format="scipy.sparse"))

## Density operators with tensor products

In [ ]:
A, B, C, D = symbols('A B C D',commutative=False)

t1 = TensorProduct(A,B,C)

d = Density([t1, 1.0])
d.doit()

t2 = TensorProduct(A,B)
t3 = TensorProduct(C,D)

d = Density([t2, 0.5], [t3, 0.5])
d.doit() 

In [ ]:
d = Density([t2+t3, 1.0])
d.doit() 

## Trace operators on density operators with spin states

In [ ]:
d = Density([JzKet(1,1),0.5],[JzKet(1,-1),0.5]);
t = Tr(d);
t

In [ ]:
t.doit()

## Partial Trace on density operators with mixed state

In [ ]:
A, B, C, D = symbols('A B C D',commutative=False)

t1 = TensorProduct(A,B,C)

d = Density([t1, 1.0])
d.doit()

t2 = TensorProduct(A,B)
t3 = TensorProduct(C,D)

d = Density([t2, 0.5], [t3, 0.5])
d

In [ ]:
tr = Tr(d,[1])
tr.doit()

## Partial trace on density operators with spin states

In [ ]:
tp1 = TensorProduct(JzKet(1,1), JzKet(1,-1))

Trace out the `0` index:

In [ ]:
d = Density([tp1,1]);
t = Tr(d,[0])
t

In [ ]:
t.doit()

Trace out the `1` index:

In [ ]:
t = Tr(d,[1])
t

In [ ]:
t.doit()

## Examples of `qapply()` on density matrices with spin states

In [ ]:
psi = Ket('psi')
phi = Ket('phi')

u = UnitaryOperator()
d = Density((psi,0.5),(phi,0.5)); d

qapply(u*d)

In [ ]:
up = JzKet(S(1)/2, S(1)/2)
down = JzKet(S(1)/2, -S(1)/2)
d = Density((up,0.5),(down,0.5))

uMat = Matrix([[0,1],[1,0]])
qapply(uMat*d)

## Example of `qapply()` on density matrices with qubits

In [ ]:
from sympy.physics.quantum.gate import UGate
from sympy.physics.quantum.qubit import Qubit

uMat = UGate((0,), Matrix([[0,1],[1,0]]))
d = Density([Qubit('0'),0.5],[Qubit('1'), 0.5])
d

In [ ]:
#after applying Not gate
qapply(uMat*d)

# Quantum Angular Momentum

This file will show how to use the various objects and methods in the `sympy.physics.quantum.spin` module, with some examples. Much of the work in this module is based off Varschalovich "Quantum Theory of Angular Momentum".

In [ ]:
from IPython.display import display

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from sympy import factor, pi, S, Sum, symbols
from sympy.physics.quantum.spin import (
    Jminus, Jx, Jz, J2, J2Op, JzKet, JzKetCoupled, Rotation, WignerD, couple, uncouple
)
from sympy.physics.quantum import (
    Dagger, hbar, qapply, represent, TensorProduct
)

## Basic spin states and operators

We can define simple spin states and operators and manipulate them with standard quantum machinery.

Define a spin ket:

In [ ]:
jz = JzKet(1,1)
jz

Find the vector representation of the state:

In [ ]:
represent(jz)

Create and evaluate an innerproduct of a bra and a ket:

In [ ]:
ip = Dagger(jz)*jz
ip

In [ ]:
ip.doit()

Apply an angular momentum operator to the state:

In [ ]:
Jz*jz

In [ ]:
qapply(Jz*jz)

In [ ]:
Jminus*jz

In [ ]:
qapply(Jminus*jz)

We can also do this for symbolic angular momentum states:

In [ ]:
j, m = symbols('j m')
jz = JzKet(j, m); jz

In [ ]:
J2*jz

In [ ]:
qapply(J2*jz)

Find the matrix representation of a angular momentum operator:

In [ ]:
represent(Jz, j=1)

## Utilizing different bases

Angular momentum states and operators can be transformed between different spin bases. We can rewrite states as states in another basis:

In [ ]:
jz = JzKet(1, 1)
jz.rewrite("Jx")

Vector representation can also be done into different bases:

In [ ]:
represent(jz, basis=Jx)

When applying operators in another spin basis, any conversion necessary to apply the state is done, then the states are given back in the original basis. So in the following example, the state returned by `qapply` are in the $J_z$ basis:

In [ ]:
Jx*jz

In [ ]:
qapply(Jx*jz)

Rewriting states and applying operators between bases can also be done symbolically. In this case, the result is given in terms of Wigner-D matrix elements (see the next section for more information on the `Rotation` operator).

In [ ]:
jz = JzKet(j, m)
jz.rewrite("Jx")

## Rotation operator

Arbitrary rotations of spin states, written in terms of Euler angles, can be modeled using the rotation operator. These methods are utilized to go between spin bases, as seen in the section above.

Define an arbitrary rotation operator. The given angles are Euler angles in the `z-y-z` convention.

In [ ]:
a, b, g = symbols('alpha beta gamma')
Rotation(a, b, g)

Find the Wigner-D matrix elements of the rotation operator as given by $\langle j, m'|\mathcal{R}(\alpha, \beta, \gamma)|j,m\rangle$:

In [ ]:
mp = symbols('mp')
r = Rotation.D(j, m, mp, a, b, g)
r

Numerical matrix elements can be evaluated using the `.doit()` method:

In [ ]:
r = Rotation.D(1, 1, 0, pi, pi/2, 0)
r

In [ ]:
r.doit()

The Wigner small-d matrix elements give rotations when $\alpha=\gamma=0$. These matrix elements can be found in the same manner as above:

In [ ]:
r = Rotation.d(j, m, mp, b)
r

In [ ]:
r = Rotation.d(1, 1, 0, pi/2)
r

In [ ]:
r.doit()

You can also directly create a Wigner-D matrix element:

In [ ]:
WignerD(j, m, mp, a, b, g)

## Coupled and uncoupled states and operators

States and operators can also written in terms of coupled or uncoupled angular momentum spaces.

### Coupled states and operators

Define a simple coupled state of two $j=1$ spin states:

In [ ]:
jzc = JzKetCoupled(1, 0, (1, 1)); jzc

Note that the Hilbert space of coupled states is the direct sum of the coupled spin spaces. This can be seen in the matrix representation of coupled states:

In [ ]:
jzc.hilbert_space

In [ ]:
represent(jzc)

We can also couple more than two spaces together. See the `JzKetCoupled` documentation for more complex coupling schemes involving more than 2 spaces.

In [ ]:
jzc = JzKetCoupled(1, 1, (S(1)/2, S(1)/2, 1))
jzc

The normal operators are assumed to be diagonal in the corresponding coupled basis:

In [ ]:
qapply(Jz*jzc)

### Uncoupled states and operators

Uncoupled states are defined as tensor products of states:

In [ ]:
jzu = TensorProduct(JzKet(1, 1), JzKet(S(1)/2, -S(1)/2)); jzu

Vector representation of tensor product states gives the vector in the direct product space:

In [ ]:
represent(jzu)

Uncoupled operators are also defined as tensor products:

In [ ]:
jzopu = TensorProduct(Jz, 1)
jzopu

In [ ]:
qapply(jzopu*jzu)

Coupled operators which are diagonalized by uncoupled states (e.g. $J_z$ and uncoupled $J_z$ eigenstates) can also be applied:

In [ ]:
qapply(Jz*jzu)

Rewriting states works as before:

In [ ]:
jzu.rewrite("Jx")

### Coulping and Uncoupling States

The `couple` method will couple an uncoupled state:

In [ ]:
jzu = TensorProduct(JzKet(1, 1), JzKet(S(1)/2, -S(1)/2))
couple(jzu)

Similarly, the uncouple method will uncouple a coupled state:

In [ ]:
jzc = JzKetCoupled(2, 1, (1, S(1)/2, S(1)/2))
uncouple(jzc)

Uncoupling can also be done with the `.rewrite` method:

In [ ]:
jzc.rewrite("Jz", coupled=False)

The `uncouple` method can also uncouple normal states if given a set of spin bases to consider:

In [ ]:
jz = JzKet(2, 1)
uncouple(jz, (1, S(1)/2, S(1)/2))

# Spin-Orbit Interaction

In [ ]:
from IPython.display import display

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from sympy import factor, pi, S, Sum, symbols
from sympy.physics.quantum.spin import (
    Jminus, Jx, Jz, J2, J2Op, JzKet, JzKetCoupled, Rotation, WignerD, couple, uncouple
)
from sympy.physics.quantum import (
    Dagger, hbar, qapply, represent, TensorProduct
)

## Symbolic calculation

If we start with a hydrogen atom, i.e. a nucleus of charge $Ze$ orbited by a single electron of charge $e$ with reduced mass $\mu$, ignoring energy from center-of-mass motion, we can write the Hamiltonian in terms of the relative momentum, $p$, and position, $r$, as:

$$H=\frac{p^2}{2\mu} - \frac{Ze^2}{r}$$

The resulting eigenfunctions have a seperate radial and angular compents, $\psi=R_{n,l}(r)Y_{l,m}(\phi,\theta)$. While the radial component is a complicated function involving Laguere polynomials, the radial part is the familiar spherical harmonics with orbital angular momentum $\vec{L}$, where $l$ and $m$ give the orbital angular momentum quantum numbers. We represent this as a angular momentum state:

In [ ]:
l, ml = symbols('l m_l')
orbit = JzKet(l, ml)
orbit

Now, the spin orbit interaction arises from the electron experiencing a magnetic field as it orbits the electrically charged nucleus. This magnetic field is:

$$\vec{B} = \frac{1}{c}\frac{Ze\vec{v}\times\vec{r}}{r^3} = \frac{Ze\vec{p}\times\vec{r}}{mcr^3}=\frac{Ze\vec{L}}{mc\hbar r^3}$$

Then the spin-orbit Hamiltonian can be written, using the electron's magnetic dipole moment $\mu$, as:

$$H_{SO} = -\vec{\mu}\cdot\vec{B} = -\left(-\frac{g\mu_B \vec{S}}{\hbar}\right)\cdot\left(\frac{Ze\vec{L}}{mc\hbar r^3}\right)$$

Ignoring the radial term:

$$\propto \vec{L}\cdot\vec{S} = J^2 - L^2 - S^2$$

for $\vec{J}$, the coupled angular momentum.

The electron spin angular momentum is given as $\vec{S}$, where the spin wavefunction is:

In [ ]:
ms = symbols('m_s')
spin = JzKet(S(1)/2, ms)
spin

From this we build our uncoupled state:

In [ ]:
state = TensorProduct(orbit, spin)
state

For clarity we will define $L^2$ and $S^2$ operators. These behave the same as `J2`, they only display differently.

In [ ]:
L2 = J2Op('L')
S2 = J2Op('S')

We also have the spin-orbit Hamiltonian:

In [ ]:
hso = J2 - TensorProduct(L2, 1) - TensorProduct(1, S2)
hso

Now we apply this to our state:

In [ ]:
apply1 = qapply(hso*state)
apply1

Note this has not applied the coupled $J^2$ operator to the states, so we couple the states and apply again:

In [ ]:
apply2 = qapply(couple(apply1))
apply2

We now collect the terms of the sum, since they share the same limits, and factor the result:

In [ ]:
subs = []
for sum_term in apply2.atoms(Sum):
    subs.append((sum_term, sum_term.function))
    limits = sum_term.limits
final = Sum(factor(apply2.subs(subs)), limits)
final

This gives us the modification of the angular part of the spin-orbit Hamiltonian. We see there is now the new $j$ quantum number in the coupled states, which we see from looking at the equation will have values $l\pm \frac{1}{2}$, and $m_j=m_l + m_s$. We still have the $l$ and $s$ quantum numbers.

# 1D Simple Harmonic Oscillator

In [ ]:
from IPython.display import display, display_pretty

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from sympy import *
from sympy.physics.quantum import *
from sympy.physics.quantum.sho1d import *
from sympy.physics.quantum.tests.test_sho1d import *

## Printing Of Operators

Create a raising and lowering operator and make sure they print correctly

In [ ]:
ad = RaisingOp('a')
ad

In [ ]:
a = LoweringOp('a')
a

In [ ]:
print(latex(ad))
print(latex(a))

In [ ]:
display_pretty(ad)
display_pretty(a)

In [ ]:
print(srepr(ad))
print(srepr(a))

In [ ]:
print(repr(ad))
print(repr(a))

## Printing of States

Create a simple harmonic state and check its printing

In [ ]:
k = SHOKet('k')
k

In [ ]:
b = SHOBra('b')
b

In [ ]:
print(pretty(k))
print(pretty(b))

In [ ]:
print(latex(k))
print(latex(b))

In [ ]:
print(srepr(k))
print(srepr(b))

## Properties

Take the dagger of the raising and lowering operators. They should return each other:

In [ ]:
Dagger(ad)

In [ ]:
Dagger(a)

Check commutators of the raising and lowering operators

In [ ]:
Commutator(ad,a).doit()

In [ ]:
Commutator(a,ad).doit()

Take a look at the dual states of the bra and ket

In [ ]:
k.dual

In [ ]:
b.dual

Taking the inner product of the bra and ket will return the Kronecker delta function

In [ ]:
InnerProduct(b,k).doit()

Take a look at how the raising and lowering operators act on states. We use qapply to apply an operator to a state

In [ ]:
qapply(ad*k)

In [ ]:
qapply(a*k)

But the states may have an explicit energy level. Let's look at the ground and first excited states

In [ ]:
kg = SHOKet(0)
kf = SHOKet(1)

In [ ]:
qapply(ad*kg)

In [ ]:
qapply(ad*kf)

In [ ]:
qapply(a*kg)

In [ ]:
qapply(a*kf)

## Number operator and Hamiltonian

Let's look at the number operator and Hamiltonian operator:

In [ ]:
k = SHOKet('k')
ad = RaisingOp('a')
a = LoweringOp('a')
N = NumberOp('N')
H = Hamiltonian('H')

The number operator is simply expressed as `ad*a`:

In [ ]:
N.rewrite('a').doit()

The number operator expressed in terms of the position and momentum operators:

In [ ]:
N.rewrite('xp').doit()

It can also be expressed in terms of the Hamiltonian operator:

In [ ]:
N.rewrite('H').doit()

The Hamiltonian operator can be expressed in terms of the raising and lowering operators, position and momentum operators, and the number operator:

In [ ]:
H.rewrite('a').doit()

In [ ]:
H.rewrite('xp').doit()

In [ ]:
H.rewrite('N').doit()

The raising and lowering operators can also be expressed in terms of the position and momentum operators

In [ ]:
ad.rewrite('xp').doit()

In [ ]:
a.rewrite('xp').doit()

### Properties

Let's take a look at how the number operator and Hamiltonian act on states:

In [ ]:
qapply(N*k)

Apply the number operator to a state returns the state times the ket:

In [ ]:
ks = SHOKet(2)
qapply(N*ks)

In [ ]:
qapply(H*k)

Let's see how the operators commute with each other:

In [ ]:
Commutator(N,ad).doit()

In [ ]:
Commutator(N,a).doit()

In [ ]:
Commutator(N,H).doit()

## Representation

We can express the operators in number operator basis. There are different ways to create a matrix in Python, we will use 3 different ways.

Sympy:

In [ ]:
represent(ad, basis=N, ndim=4, format='sympy')

Numpy:

In [ ]:
represent(ad, basis=N, ndim=5, format='numpy')

`scipy.sparse`:

In [ ]:
sparse_rep = represent(ad, basis=N, ndim=4, format='scipy.sparse', spmatrix='lil')
sparse_rep

In [ ]:
print(sparse_rep)

The same can be done for the other operators

In [ ]:
represent(a, basis=N, ndim=4, format='sympy')

In [ ]:
represent(N, basis=N, ndim=4, format='sympy')

In [ ]:
represent(H, basis=N, ndim=4, format='sympy')

Bras and kets can also be represented:

In [ ]:
k0 = SHOKet(0)
k1 = SHOKet(1)
b0 = SHOBra(0)
b1 = SHOBra(1)

In [ ]:
represent(k0, basis=N, ndim=5, format='sympy')

In [ ]:
represent(k1, basis=N, ndim=5, format='sympy')

In [ ]:
represent(b0, basis=N, ndim=5, format='sympy')

In [ ]:
represent(b1, basis=N, ndim=5, format='sympy')

# Symbolic Quantum Computing - qubits

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from sympy import sqrt, symbols, Rational
from sympy import expand, Eq, Symbol, simplify, exp, sin, srepr
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import *
from sympy.physics.quantum.gate import *
from sympy.physics.quantum.grover import *
from sympy.physics.quantum.qft import QFT, IQFT, Fourier
from sympy.physics.quantum.circuitplot import circuit_plot

## Qubits

In [ ]:
alpha, beta = symbols('alpha beta', real=True)

In [ ]:
psi = alpha*Qubit('00') + beta*Qubit('11'); psi

In [ ]:
Dagger(psi)

In [ ]:
qapply(Dagger(Qubit('00'))*psi)

In [ ]:
for state, prob in measure_all(psi):
    display(state)
    display(prob)

Qubits can be represented in the computational basis.

In [ ]:
represent(psi)

## Gates

Gate objects are the operators which act on a quantum state.

In [ ]:
g = X(0)
g

In [ ]:
represent(g, nqubits=2)

In [ ]:
c = H(0)*Qubit('00')
c

In [ ]:
qapply(c)

In [ ]:
for gate in [H,X,Y,Z,S,T]:
    for state in [Qubit('0'),Qubit('1')]:
        lhs = gate(0)*state
        rhs = qapply(lhs)
        display(Eq(lhs,rhs))

<h2>Symbolic gate rules and circuit simplification</h2>

In [ ]:
for g1 in (Y,Z,H):
    for g2 in (Y,Z,H):
        e = Commutator(g1(0),g2(0))
        if g1 != g2:
            display(Eq(e,e.doit()))

In [ ]:
c = H(0)*X(1)*H(0)**2*CNOT(0,1)*X(1)**3*X(0)*Z(1)**2
c

In [ ]:
circuit_plot(c, nqubits=2);

This performs a commutator/anticommutator aware bubble sort algorithm to simplify a circuit:

In [ ]:
gate_simp(c)

In [ ]:
circuit_plot(gate_simp(c),nqubits=2)

# Gate Decomposition

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from sympy import sqrt, symbols, Rational
from sympy import expand, Eq, Symbol, simplify, exp, sin
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import *
from sympy.physics.quantum.gate import *
from sympy.physics.quantum.grover import *
from sympy.physics.quantum.qft import QFT, IQFT, Fourier
from sympy.physics.quantum.circuitplot import circuit_plot

<h2>Example 1</h2>

Create a symbolic controlled-Y gate

In [ ]:
CY10 = CGate(1, Y(0));
CY10

Decompose it into elementary gates and plot it

In [ ]:
CY10.decompose()

In [ ]:
circuit_plot(CY10.decompose(), nqubits=2);

<h2>Example 2</h2>

Create a controlled-Z gate

In [ ]:
CZ01 = CGate(0, Z(1));
CZ01

Decompose and plot it

In [ ]:
CZ01.decompose()

In [ ]:
circuit_plot(CZ01.decompose(), nqubits=2);

<h2>Example 3</h2>

Create a SWAP gate

In [ ]:
SWAP10 = SWAP(1, 0);
SWAP10

Decompose and plot it

In [ ]:
SWAP10.decompose()

In [ ]:
circuit_plot(SWAP10.decompose(), nqubits=2);

<h2>All together now</h2>

In [ ]:
gates = [CGate(1,Y(0)), CGate(0,Z(1)), SWAP(1, 0)]

In [ ]:
for g in gates:
    dg = g.decompose()
    display(Eq(g, dg))
    circuit_plot(g, nqubits=2)
    circuit_plot(dg, nqubits=2)    

# Fidelity of quantum states

https://en.wikipedia.org/wiki/Fidelity_of_quantum_states

In [ ]:
from IPython.display import display

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from sympy import *
from sympy.physics.quantum import *
from sympy.physics.quantum.density import *
from sympy.physics.quantum.spin import (
    Jx, Jy, Jz, Jplus, Jminus, J2,
    JxBra, JyBra, JzBra,
    JxKet, JyKet, JzKet,
)
from IPython.core.display import display_pretty
from sympy.physics.quantum.operator import *

## Basic fidelity using spin kets

In [ ]:
up = JzKet(S(1)/2,S(1)/2)
down = JzKet(S(1)/2,-S(1)/2)
amp = 1/sqrt(2)
updown = (amp * up ) + (amp * down)
updown

Using `represent` turns the kets into matrices:

In [ ]:
up_dm = represent(up * Dagger(up))
down_dm = represent(down * Dagger(down)) 
updown_dm = represent(updown * Dagger(updown))
updown_dm

Another entangled state:

In [ ]:
updown2 = (sqrt(3)/2 )* up + (1/2)*down
updown2

In [ ]:
fidelity(up_dm, up_dm)

In [ ]:
fidelity(up_dm, down_dm)

In [ ]:
fidelity(up_dm, updown_dm).evalf()

Alternatively, put kets into density operator and compute fidelity:

In [ ]:
d1 = Density( [updown, 0.25], [updown2, 0.75])
d2 = Density( [updown, 0.75], [updown2, 0.25])
fidelity(d1, d2)

## Fidelity with qubit states

In [ ]:
from sympy.physics.quantum.qubit import Qubit
state1 = Qubit('0')
state2 = Qubit('1')
state3 = (1/sqrt(2))*state1 + (1/sqrt(2))*state2
state4 = (sqrt(S(2)/3))*state1 + (1/sqrt(3))*state2

In [ ]:
state3

In [ ]:
state4

In [ ]:
state1_dm = Density([state1, 1])
state2_dm = Density([state2, 1])
state3_dm = Density([state3, 1])

In [ ]:
d1 = Density([state3, 0.70], [state4, 0.30])
d2 = Density([state3, 0.20], [state4, 0.80])
fidelity(d1, d2)

# Teleportation

https://en.wikipedia.org/wiki/Quantum_teleportation

In [ ]:
%matplotlib inline

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from sympy import sqrt, symbols, Rational
from sympy import expand, Eq, Symbol, simplify, exp, sin
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import *
from sympy.physics.quantum.gate import *
from sympy.physics.quantum.grover import *
from sympy.physics.quantum.qft import QFT, IQFT, Fourier
from sympy.physics.quantum.circuitplot import circuit_plot

## Teleportation

In [ ]:
a,b = symbols('a b', real=True)
state = Qubit('000')*a + Qubit('001')*b
state

In [ ]:
entangle1_2 = CNOT(1,2)*HadamardGate(1)
entangle1_2

In [ ]:
state = qapply(entangle1_2*state)
state

In [ ]:
entangle0_1 = HadamardGate(0)*CNOT(0,1)
entangle0_1

In [ ]:
circuit_plot(entangle0_1*entangle1_2, nqubits=3);

In [ ]:
state = qapply(entangle0_1*state)
state

In [ ]:
result = measure_partial(state, (0,1))

In [ ]:
state = (result[2][0]*2).expand()
state

In [ ]:
state = qapply(XGate(2)*state)
state

# Dense Coding

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from sympy import sqrt, symbols, Rational
from sympy import expand, Eq, Symbol, simplify, exp, sin
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import *
from sympy.physics.quantum.gate import *
from sympy.physics.quantum.grover import *
from sympy.physics.quantum.qft import QFT, IQFT, Fourier
from sympy.physics.quantum.circuitplot import circuit_plot

## Dense coding

In [ ]:
psi = Qubit('00')/sqrt(2) + Qubit('11')/sqrt(2); psi

In [ ]:
circuits = [H(1)*CNOT(1,0), H(1)*CNOT(1,0)*X(1), H(1)*CNOT(1,0)*Z(1), H(1)*CNOT(1,0)*Z(1)*X(1)]

In [ ]:
for circuit in circuits:
    circuit_plot(circuit, nqubits=2)
    display(Eq(circuit*psi,qapply(circuit*psi)))

# Quantum Error Correction

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from sympy import sqrt, symbols, Rational
from sympy import expand, Eq, Symbol, simplify, exp, sin
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import *
from sympy.physics.quantum.gate import *
from sympy.physics.quantum.grover import *
from sympy.physics.quantum.qft import QFT, IQFT, Fourier
from sympy.physics.quantum.circuitplot import circuit_plot

<h2>5 qubit code</h2>

In [ ]:
M0 = Z(1)*X(2)*X(3)*Z(4)
M0

In [ ]:
M1 = Z(2)*X(3)*X(4)*Z(0)
M1

In [ ]:
M2 = Z(3)*X(4)*X(0)*Z(1)
M2

In [ ]:
M3 = Z(4)*X(0)*X(1)*Z(2)
M3

These operators should mutually commute.

In [ ]:
gate_simp(Commutator(M0,M1).doit())

And square to the identity.

In [ ]:
for o in [M0,M1,M2,M3]:
    display(gate_simp(o*o))

<h2>Codewords</h2>

In [ ]:
zero = Rational(1,4)*(1+M0)*(1+M1)*(1+M2)*(1+M3)*IntQubit(0, 5)
zero

In [ ]:
qapply(4*zero)

In [ ]:
one = Rational(1,4)*(1+M0)*(1+M1)*(1+M2)*(1+M3)*IntQubit(2**5-1, 5)
one

In [ ]:
qapply(4*one)

<h2>The encoding circuit</h2>

In [ ]:
encoding_circuit = H(3)*H(4)*CNOT(2,0)*CNOT(3,0)*CNOT(4,0)*H(1)*H(4)*\
                   CNOT(2,1)*CNOT(4,1)*H(2)*CNOT(3,2)*CNOT(4,2)*H(3)*\
                   H(4)*CNOT(4, 3)*Z(4)*H(4)*Z(4)

In [ ]:
circuit_plot(encoding_circuit, nqubits=5, scale=0.5);

In [ ]:
represent(4*encoding_circuit, nqubits=5)

# Quantum Fourier Transform

https://en.wikipedia.org/wiki/Quantum_Fourier_transform

In [ ]:
%matplotlib inline

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from ipywidgets import interact, interactive
from IPython.display import clear_output, display, HTML, Audio

In [ ]:
from sympy import sqrt, symbols, Rational
from sympy import expand, Eq, Symbol, simplify, exp, sin
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import *
from sympy.physics.quantum.gate import *
from sympy.physics.quantum.grover import *
from sympy.physics.quantum.qft import QFT, IQFT, Fourier
from sympy.physics.quantum.circuitplot import circuit_plot

## QFT Gate and Circuit

The Quantum Fourier Transform (QFT) is useful for a quantum algorithm for factoring numbers which is exponentially faster than what is thought to be possible on a classical machine. The transform does a DFT on the state of a quantum system. There is a simple decomposition of the QFT in terms of a few elementary gates.

Build a 3 qubit QFT and decompose it into primitive gates:

In [ ]:
fourier = QFT(0,3).decompose()
fourier

In [ ]:
circuit_plot(fourier, nqubits=3);

In [ ]:
def plot_qft(n):
    circuit_plot(QFT(0,n).decompose(), nqubits=n)

In [ ]:
interact(plot_qft, n=(2,8));

The QFT circuit can be represented in various symbolic forms.

In [ ]:
m = represent(QFT(0,3), nqubits=3)
m

In [ ]:
represent(Fourier(0,3), nqubits=3)*4/sqrt(2)

## QFT in action

Build a 3 qubit state to take the QFT of:

In [ ]:
state = (Qubit('000') + Qubit('010') + Qubit('100') + Qubit('110'))/sqrt(4)
state

Perform the QFT:

In [ ]:
qapply(fourier*state)

In [ ]:
def apply_qft(n):
    state = Qubit(IntQubit(n, 3))
    result = qapply(QFT(0,3).decompose()*state)
    display(state)
    display(result)

In [ ]:
interact(apply_qft, n=(0,7));

# Grover's Algorithm

https://en.wikipedia.org/wiki/Grover%27s_algorithm

In [ ]:
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
from sympy import init_printing
init_printing(use_latex=True)

In [ ]:
from sympy import sqrt, symbols, Rational
from sympy import expand, Eq, Symbol, simplify, exp, sin
from sympy.physics.quantum import *
from sympy.physics.quantum.qubit import *
from sympy.physics.quantum.gate import *
from sympy.physics.quantum.grover import *
from sympy.physics.quantum.qft import QFT, IQFT, Fourier
from sympy.physics.quantum.circuitplot import circuit_plot

## Grover's algorithm for 3 qubits

In [ ]:
nqubits = 3

Grover's algorithm is a quantum algorithm which searches an unordered database (inverts a function). It provides polynomial speedup over classical brute-force search ($O(\sqrt{N}) vs. O(N))$ 

Define a black box function that returns True if it is passed the state we are searching for.

In [ ]:
def black_box(qubits):
    return True if qubits == IntQubit(1, qubits.nqubits) else False

Build a uniform superposition state to start the search.

In [ ]:
psi = superposition_basis(nqubits)
psi

In [ ]:
v = OracleGate(nqubits, black_box)

Perform two iterations of Grover's algorithm.  Each iteration, the amplitude of the target state increases.

In [ ]:
iter1 = qapply(grover_iteration(psi, v))
iter1

In [ ]:
iter2 = qapply(grover_iteration(iter1, v))
iter2

A single shot measurement is performed to retrieve the target state.

In [ ]:
measure_all_oneshot(iter2)